# Analisando um jogador especifico



In [1]:
from statsbombpy import sb
import pandas as pd
from mplsoccer import VerticalPitch,Pitch
from highlight_text import ax_text, fig_text
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
import seaborn as sns
from functools import reduce

In [3]:
euro_2024_matches = sb.matches(competition_id=55, season_id=282)

In [12]:
# função para pegar as partidas de um time específico

def team_matches(team, df_matches):
    team_matches = df_matches.query((f'home_team == "{team}" or away_team == "{team}"'))
    return team_matches.sort_values(by='match_date', ascending=False)

In [13]:
team = "Netherlands"
df_matches = sb.matches(competition_id=55, season_id=282) # partidas da eurocopa 2024

team_matches = team_matches(team, df_matches)

In [7]:
team_matches

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3942819,2024-07-10,22:00:00.000,Europe - UEFA Euro,2024,Netherlands,England,1,2,available,...,2024-07-15T07:57:36.684453,6,Semi-finals,Signal-Iduna-Park,Felix Zwayer,Ronald Koeman,Gareth Southgate,1.1.0,2,2
3,3942382,2024-07-06,22:00:00.000,Europe - UEFA Euro,2024,Netherlands,Turkey,2,1,available,...,2024-07-10T06:56:35.146850,5,Quarter-finals,Olympiastadion Berlin,Clément Turpin,Ronald Koeman,Vincenzo Montella,1.1.0,2,2
12,3941021,2024-07-02,19:00:00.000,Europe - UEFA Euro,2024,Romania,Netherlands,0,3,available,...,2024-07-12T23:28:27.122555,4,Round of 16,Allianz Arena,Felix Zwayer,Edward Iordănescu,Ronald Koeman,1.1.0,2,2
5,3930180,2024-06-25,19:00:00.000,Europe - UEFA Euro,2024,Netherlands,Austria,2,3,available,...,2024-07-11T13:25:37.289456,3,Group Stage,Olympiastadion Berlin,Ivan Kružliak,Ronald Koeman,Ralf Rangnick,1.1.0,2,2
22,3930173,2024-06-21,22:00:00.000,Europe - UEFA Euro,2024,Netherlands,France,0,0,available,...,2024-07-12T19:39:43.381031,2,Group Stage,Trainingszentrum RB Leipzig Platz 1,Anthony Taylor,Ronald Koeman,Didier Deschamps,1.1.0,2,2
29,3938637,2024-06-16,16:00:00.000,Europe - UEFA Euro,2024,Poland,Netherlands,1,2,available,...,2024-07-12T03:39:53.010211,1,Group Stage,Volksparkstadion,Artur Manuel Ribeiro Soares Dias,Michał Probierz,Ronald Koeman,1.1.0,2,2


### Eventos da partida

In [23]:
# eventos da partida Holanda x Turquia
match = sb.events(match_id=3942382)

# função para tratar eventos da partida
def split_locations(df):
    df[['x', 'y']] = df['location'].apply(pd.Series)
    df[['pass_end_x', 'pass_end_y']] = df['pass_end_location'].apply(pd.Series) # separar ponto final dos passes
    df[['carry_end_x', 'carry_end_y']] = df['carry_end_location'].apply(pd.Series) # separar ponto final das conduções
    return df

In [24]:
match_events = split_locations(match)

In [25]:
match_events.columns

Index(['50_50', 'bad_behaviour_card', 'ball_receipt_outcome',
       'ball_recovery_recovery_failure', 'block_deflection',
       'block_save_block', 'carry_end_location', 'clearance_aerial_won',
       'clearance_body_part', 'clearance_head', 'clearance_left_foot',
       'clearance_right_foot', 'counterpress', 'dribble_nutmeg',
       'dribble_outcome', 'duel_outcome', 'duel_type', 'duration',
       'foul_committed_advantage', 'foul_committed_card',
       'foul_committed_offensive', 'foul_committed_type', 'foul_won_advantage',
       'foul_won_defensive', 'goalkeeper_body_part', 'goalkeeper_end_location',
       'goalkeeper_outcome', 'goalkeeper_position',
       'goalkeeper_shot_saved_to_post', 'goalkeeper_technique',
       'goalkeeper_type', 'id', 'index', 'injury_stoppage_in_chain',
       'interception_outcome', 'location', 'match_id', 'minute',
       'miscontrol_aerial_won', 'off_camera', 'out', 'pass_aerial_won',
       'pass_angle', 'pass_assisted_shot_id', 'pass_body_part

### Agregações de eventos dos jogadores

In [27]:
match_events

,50_50,bad_behaviour_card,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,block_save_block,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,...,team_id,timestamp,type,under_pressure,x,y,pass_end_x,pass_end_y,carry_end_x,carry_end_y
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,941,00:00:00.000,Starting XI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,909,00:00:00.000,Starting XI,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,941,00:00:00.000,Half Start,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,909,00:00:00.000,Half Start,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,909,00:00:00.000,Half Start,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3410,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,941,00:13:31.083,Referee Ball-Drop,NaN,105.5,38.4,NaN,NaN,NaN,NaN
3411,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,909,00:30:27.841,Own Goal Against,NaN,5.6,47.4,NaN,NaN,NaN,NaN
3412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,941,00:30:27.841,Own Goal For,NaN,114.5,32.7,NaN,NaN,NaN,NaN
3413,NaN,Yellow Card,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,909,00:47:53.463,Bad Behaviour,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
match_events['team'].unique()

array(['Netherlands', 'Turkey'], dtype=object)

In [57]:
# Criando uma lista de jogadores da equipe analisada
team = "Netherlands"

players_team = [match_events.query(f'team == "{team}"')['player'].unique()]

players_team = list(players_team[0][1:])

In [59]:
player_name = 'Xavi Simons'
events_types = ['Ball Receipt*']
player_ball_rec=match_events[(match_events.player==player_name)&(match_events.type.isin(events_types))]


In [ ]:
# continuar amanhã armazenando a soma de cada tipo de evento em um dataframe